# Setup

In [35]:
import pandas as ps
from datetime import datetime
from neo4j import GraphDatabase, RoutingControl

# Extract

In [29]:
df_org = pd.read_excel("VehicleData.xlsx")

In [40]:
df_org["SOP"] = df_org["SOP"].apply(lambda r: datetime.strptime(r,"%Y-%m"))

In [60]:
df_org.head()

,Bosch ID,Source ID,FASKey,Brand,Vehicle Series,Model code,Vehicle Type,Engine Code,EOP,SOP,...,Market,Body Type,CC,Drive Form,Cylinders,Engine Type,Engine Aspiration,Engine Carburation,Horse Power,Vehicle Class
0,007MY000000000002323,17009,AUD0001446,AUDI,A3,8P1,A3 2.0,BVZ,2008-06,2005-11-01,...,Malaysia,Hatchback,2.0,Front-wheel drive,4.0,Petrol,Naturally Aspirated,Direct Injection,110,PC
1,007MY000000000002336,150793,AUD0142881,AUDI,Q8 E-Tron,NaN,Q8 E-Tron Quattro,EAVA;EATA;EAUA,NaN,2024-01-01,...,Malaysia,SUV,EV,All-wheel drive,NaN,Accumulator/Electric,Not applicable,Accumulator/Electric,370,PC
2,007MY000000000002344,128606,ROL0000081,ROLLS-ROYCE,Phantom,NaN,Phantom 6.75,N74 B68 A,NaN,2024-01-01,...,Malaysia,Saloon,6.75,Rear-wheel drive,12.0,Petrol,NaN,Direct Injection,420,PC
3,007MY000000000002347,147328,SUB0131210,SUBARU,WRX,NaN,WRX 2.4 4x4,FA24F,NaN,2022-03-01,...,Malaysia,Saloon,2.4,All-wheel drive,4.0,Petrol,NaN,Direct Injection,202,PC
4,007MY000000000002356,138698,BMW0107748,BMW,X3 Series,G01,X3 xDrive 30e 2.0,B48B20A,NaN,2019-12-01,...,Malaysia,SUV,2.0,All-wheel drive,4.0,Plug-in Hybrid/petrol,Turbocharged,Direct Injection,215,PC


# Relationship - Brand, Model

In [65]:
df_brand_models = df_org[['Bosch ID','Brand','Vehicle Type','SOP']].drop_duplicates()
with GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "neo4j!!!!")) as driver:
    for i in range(len(df_brand_models)):
        try:
            r = df_brand_models.loc[i]
    
            driver.execute_query(
                "MERGE (b:Brand {brand: $brand}) "
                "MERGE (m:Model {model_name: $model, bosch_id: $bosch_id, sop_date: date() }) "
                "MERGE (b)-[:MAKES]->(m)",
                brand=r["Brand"], model=r["Vehicle Type"], bosch_id=r["Bosch ID"], database_="neo4j",
            )
        
        except Exception as e:
            print(e)
        

735
1380
1975
2116


In [69]:
with GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "neo4j!!!!")) as driver:
    for i in range(len(df_brand_models)):
        try:
            r = df_brand_models.loc[i]
    
            driver.execute_query(
                "MATCH (m:Model {bosch_id: $bosch_id}) "
                "SET m.sop_date = date($sop_date)",
                sop_date=r["SOP"], bosch_id=r["Bosch ID"], database_="neo4j",
            )
        
        except Exception as e:
            print(e)

735
1380
1975
2116


# Model, Engine Type, Transmission

In [77]:
df = df_org[['Bosch ID', 'Vehicle Type', 'Engine Type', 'Transmission']].drop_duplicates(subset=['Transmission'])
with GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "neo4j!!!!")) as driver:
    for i in range(len(df)):
        try:
            r = df.loc[i]
    
            driver.execute_query(
                "MATCH (m:Model {bosch_id: $bosch_id}) "
                "MERGE (e:EngineType {engineType: $eng}) "
                "MERGE (t:Transmission {transmission_type: $tx}) "
                "MERGE (e)-[:POWERS]-(m) "
                "MERGE (t)-[:DRIVES]-(m) "
                ,
                eng=r['Engine Type'], model=r["Vehicle Type"], tx=r['Transmission'], bosch_id=r["Bosch ID"], database_="neo4j",
            )
        
        except Exception as e:
            print(e)

{code: Neo.ClientError.Statement.SemanticError} {message: Cannot merge the following node because of NaN property value for 'transmission_type': (:Transmission {transmission_type: NaN})}
5
6
7
8
9
10
11
12
14
15
18
19
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
41
42
43
44
45
46
47
48
49
50
51
52
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [54]:
sop_years = list(set(list(df_org["SOP"].apply(lambda r: r.year))))
sop_years.sort()

In [56]:
with GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "neo4j!!!!")) as driver:
    for i in range(len(df)):
        try:
            r = df.loc[i]
    
            driver.execute_query(
                "MATCH (e:EngineType {engineType: $eng}) "
                "MERGE (m:Model {model_name: $model}) "
                "MERGE (t:Transmission {transmission_type: $tx}) "
                "MERGE (e)-[:POWERS]->(m) "
                "MERGE (t)-[:DRIVES]->(m) "
                ,
                eng=r['Engine Type'], model=r["Vehicle Type"], tx=r['Transmission'], database_="neo4j",
            )
        
        except Exception as e:
            pass

[1977,
 1978,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]